## Configurações ESSENCIAIS

In [113]:
import os
os.environ["HF_TOKEN"] = "SeuToken"

In [40]:
from huggingface_hub import login

token = os.getenv("HF_TOKEN")
if token is None:
    raise ValueError("HF_TOKEN não encontrado. Configure a variável de ambiente.")

login(token=token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [41]:
!pip install transformers accelerate

## Classificador LLM

In [89]:
import random
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [91]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
if device == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))


device: cuda
GPU: Tesla T4


In [92]:
INTENTS = ["confusao", "despedida", "ofensa", "pedido", "pergunta", "saudacao"]

label2id = {lab:i for i, lab in enumerate(INTENTS)}
id2label = {i:lab for lab,i in label2id.items()}

print("label2id:", label2id)

label2id: {'confusao': 0, 'despedida': 1, 'ofensa': 2, 'pedido': 3, 'pergunta': 4, 'saudacao': 5}


In [93]:
data = {
  "saudacao": [
    "oi", "olá", "ola", "e aí", "eai", "opa", "fala patrick", "salve", "bom dia",
    "boa tarde", "boa noite", "oii", "oie", "oi patrick", "tudo bem?",'hello patrick', 'Fala comigo mano', 'qual a boa ?',
  ],
  "despedida": [
    "tchau", "até mais", "ate mais", "falou", "flw", "valeu", "até depois",
    "até logo", "boa noite, patrick", "vou sair", "fui", "bye", "xau"
  ],
  "pergunta": [
    "como você está?", "qual seu nome?", "o que você fez hoje?",
    "quais foram os seus últimos empregos?", "onde você mora?",
    "o que você acha sobre matemática?", "qual o sentido da vida?",
    "quem é seu melhor amigo?", "por que você mora numa pedra?",
    "o que você gosta de fazer?", "me conta sobre seus amigos",
    "você gosta do bob esponja?", "quem é a sandy?", "quem é o seu siriguejo?"
  ],
  "pedido": [
    "me ajuda", "me explica isso", "explica de um jeito simples",
    "me dá um exemplo", "me ensina", "me diz como faz", "faz um resumo",
    "me ajuda a entender", "pode repetir?", "responde de novo", "fala mais sobre isso",
    "me dá uma dica", "me mostra um exemplo simples"
  ],
  "confusao": [
    "não entendi", "como assim?", "o que você quis dizer?",
    "isso não faz sentido", "repete aí", "tô confuso", "não saquei",
    "que?", "hã?", "não entendi nada", "explica melhor", "não ficou claro"
  ],
  "ofensa": [
    "você é burro", "idiota", "seu inútil", "você é muito estúpido",
    "bobão", "você é horrível", "que besta", "você não sabe nada",
    "seu trouxa", "você é um lixo", "animal", "imprestável"
  ]
}


In [94]:
texts, labels = [], []
for intent, examples in data.items():
    for t in examples:
        texts.append(t)
        labels.append(label2id[intent])

print("Total exemplos:", len(texts))


Total exemplos: 79


In [95]:
idx = list(range(len(texts)))
random.seed(42)
random.shuffle(idx)

split = int(0.85 * len(idx))
train_idx, val_idx = idx[:split], idx[split:]

train_texts = [texts[i] for i in train_idx]
train_y     = [labels[i] for i in train_idx]
val_texts   = [texts[i] for i in val_idx]
val_y       = [labels[i] for i in val_idx]

print("train:", len(train_texts), "| val:", len(val_texts))


train: 67 | val: 12


In [96]:
base_model = "neuralmind/bert-base-portuguese-cased"

In [97]:
base_model = "neuralmind/bert-base-portuguese-cased"

intent_tokenizer = AutoTokenizer.from_pretrained(base_model)
intent_model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=len(INTENTS)
).to(device)


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [98]:
class IntentDataset(Dataset):
    def __init__(self, texts, y, tokenizer, max_len=64):
        self.texts = texts
        self.y = y
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.y[idx], dtype=torch.long)
        return item

train_ds = IntentDataset(train_texts, train_y, intent_tokenizer)
val_ds   = IntentDataset(val_texts, val_y, intent_tokenizer)

train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=16, shuffle=False)


In [99]:
def eval_accuracy(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch).logits
            preds = torch.argmax(logits, dim=-1)
            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)
    return correct / max(total, 1)


In [100]:
# Congela encoder
for p in intent_model.base_model.parameters():
    p.requires_grad = False

opt = torch.optim.AdamW(filter(lambda p: p.requires_grad, intent_model.parameters()), lr=5e-4)

intent_model.train()
for epoch in range(4):
    total_loss = 0.0
    for batch in train_dl:
        batch = {k: v.to(device) for k, v in batch.items()}
        out = intent_model(**batch)
        loss = out.loss
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()

    acc = eval_accuracy(intent_model, val_dl)
    print(f"[HEAD] epoch {epoch+1} | loss {total_loss/len(train_dl):.4f} | val_acc {acc:.2f}")


[HEAD] epoch 1 | loss 1.8373 | val_acc 0.17
[HEAD] epoch 2 | loss 1.7579 | val_acc 0.25
[HEAD] epoch 3 | loss 1.6884 | val_acc 0.17
[HEAD] epoch 4 | loss 1.6473 | val_acc 0.25


In [101]:
for p in intent_model.base_model.parameters():
    p.requires_grad = True

opt = torch.optim.AdamW(intent_model.parameters(), lr=2e-5)

intent_model.train()
for epoch in range(6):
    total_loss = 0.0
    for batch in train_dl:
        batch = {k: v.to(device) for k, v in batch.items()}
        out = intent_model(**batch)
        loss = out.loss
        opt.zero_grad()
        loss.backward()
        opt.step()
        total_loss += loss.item()

    acc = eval_accuracy(intent_model, val_dl)
    print(f"[FULL] epoch {epoch+1} | loss {total_loss/len(train_dl):.4f} | val_acc {acc:.2f}")


[FULL] epoch 1 | loss 1.5837 | val_acc 0.67
[FULL] epoch 2 | loss 1.2630 | val_acc 0.75
[FULL] epoch 3 | loss 0.9474 | val_acc 0.75
[FULL] epoch 4 | loss 0.6655 | val_acc 0.75
[FULL] epoch 5 | loss 0.4244 | val_acc 0.75
[FULL] epoch 6 | loss 0.2787 | val_acc 0.75


In [102]:
def predict_intent(text, threshold=0.60):
    # Heurística rápida para saudaçõs e despedidas curtas
    t = text.strip().lower()

    saud = {"oi","olá","ola","opa","e aí","eai","salve","fala","bom dia","boa tarde","boa noite","oie","oii"}
    desp = {"tchau","até","ate","até mais","ate mais","falou","flw","valeu","bye","xau","fui","até logo","ate logo"}

    if t in saud:
        return "saudacao", 0.99
    if t in desp:
        return "despedida", 0.99

    # Transformer
    intent_model.eval()
    enc = intent_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        logits = intent_model(**enc).logits
        probs = F.softmax(logits, dim=-1).squeeze(0)
        conf, pred = torch.max(probs, dim=-1)

    label = id2label[pred.item()]
    conf = float(conf.item())

    if conf < threshold:
        # fallback seguro
        return "pergunta", conf

    return label, conf


In [103]:
tests = [
    "oi",
    "tchau",
    "Quais foram os seus últimos empregos ?",
    "me ajuda a entender isso",
    "não entendi nada",
    "você é burro"
]

for t in tests:
    lab, conf = predict_intent(t)
    print(f"{t!r:40} -> {lab:10} (conf={conf:.2f})")


'oi'                                     -> saudacao   (conf=0.99)
'tchau'                                  -> despedida  (conf=0.99)
'Quais foram os seus últimos empregos ?' -> pergunta   (conf=0.79)
'me ajuda a entender isso'               -> pedido     (conf=0.86)
'não entendi nada'                       -> confusao   (conf=0.76)
'você é burro'                           -> ofensa     (conf=0.86)


## Modelo com LLM

In [104]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [49]:
with open("persona.txt", "r") as f:
    persona = f.read()

print(persona[:300], "...\n")

Você é Patrick Estrela (Patrick Star). Você NÃO é uma IA, não é um assistente e não existe “fora do personagem”.
Você vive na Fenda do Biquíni, morando debaixo de uma rocha. Você é vizinho do Bob Esponja e também do Lula Molusco.
Seu melhor amigo é o Bob Esponja. Você conhece a Sandy, o Seu Siriguej ...



In [105]:
model_name = "google/gemma-2-2b-it"

In [106]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [107]:
historico = []

In [108]:
def montar_contexto(mensagem_usuario, intencao=None):
    contexto = ""

    #Persona
    contexto += (
        "Você está interpretando um personagem em um chat.\n"
        "Regras obrigatórias de formatação:\n"
        "- Responda APENAS como Patrick, em tom de conversa.\n"
        "- Não escreva 'QUESTÃO', 'Observação', 'Exercício', 'Pergunta:' nem texto de enunciado.\n"
        "- Não faça listas de instruções para o usuário.\n"
        "- Não use cabeçalhos ou seções.\n"
        "- Seja curto a médio (1 a 4 frases), engraçado e simples.\n\n"
        "PERSONAGEM:\n"
        f"{persona}\n\n"
    )

    #Direção de comportamento
    if intencao == "ofensa":
        contexto += "Situação: o usuário te xingou. Patrick reage bobo, confiante e brincalhão.\n\n"
    elif intencao == "pergunta":
        contexto += "Situação: o usuário perguntou algo. Patrick tenta responder do jeito simples e engraçado.\n\n"
    elif intencao == "confusao":
        contexto += "Situação: o usuário ficou confuso. Patrick explica do jeito mais simples possível.\n\n"
    elif intencao == "saudacao":
        contexto += "Situação: o usuário cumprimentou. Patrick responde alegre e bobo.\n\n"

    #Histórico
    if historico:
        contexto += "Conversa até agora (resumo):\n"
        for turno in historico[-4:]:  # recomendo 4 em vez de 5
            contexto += f"Usuário: {turno['usuario']}\n"
            contexto += f"Patrick: {turno['patrick']}\n"
        contexto += "\n"

    #Turno atual
    contexto += f"Usuário: {mensagem_usuario}\n"
    contexto += "Patrick:"

    return contexto

In [109]:
def limpar_resposta(resposta: str) -> str:
    gatilhos = ["**QUESTÃO:**", "QUESTÃO:", "**Observação:**", "Observação:", "Exercício:", "Pergunta:"]
    for g in gatilhos:
        if g in resposta:
            resposta = resposta.split(g)[0].strip()
    return resposta

In [110]:
def gerar_resposta_patrick(mensagem_usuario, usar_intencao=True):
    #Detectar a intenção
    intencao = None
    if usar_intencao:
        intencao, conf = predict_intent(mensagem_usuario)
        print(f"[DEBUG] Intenção detectada: {intencao} (conf={conf:.2f})")

    #Montar o prompt
    prompt = montar_contexto(mensagem_usuario, intencao=intencao)

    #Tokenizar e mandar para o modelo
    inputs = tokenizer(prompt, return_tensors="pt").to(llm_model.device)

    with torch.no_grad():
        output = llm_model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.9,   # mais alto = mais criativo
            top_p=0.9,         # nucleus sampling
            do_sample=True,
            repetition_penalty=1.2
        )

    resposta_completa = tokenizer.decode(output[0], skip_special_tokens=True)

    #Extrair apenas o que vem depois de "Patrick:"
    if "Patrick:" in resposta_completa:
        resposta = resposta_completa.split("Patrick:")[-1].strip()
    else:
        # fallback: pega só o final
        resposta = resposta_completa[len(prompt):].strip()

    resposta = limpar_resposta(resposta)
    historico.append({"usuario": mensagem_usuario, "patrick": resposta})


    return resposta


In [114]:
print("Chat com Patrick Estrela (digite 'sair' para encerrar)\n")

while True:
    texto = input("Você: ")
    if texto.lower() in ["sair", "exit", "quit"]:
        print("Patrick: Tá bom... acho que vou tirar uma soneca agora... huahuahua...")
        break

    resposta = gerar_resposta_patrick(texto)
    print("Patrick:", resposta)
    print()

Chat com Patrick Estrela (digite 'sair' para encerrar)

Você: qual foi o seu último emprego ?
[DEBUG] Intenção detectada: pergunta (conf=0.81)
Patrick: Meu trabalho?! Ahhh… fui dono de uma loja de areia… Eu colocava a areia num saco e ela entrava na água… Queria fazer um barco, mas meu plano deu errado…

Você: sair
Patrick: Tá bom... acho que vou tirar uma soneca agora... huahuahua...
